In [1]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install torch
!{sys.executable} -m pip install rdkit


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 43.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [3]:
device = torch.device("cpu")

In [4]:
# Define a Reaction dataset class
class ReactionDataset(Dataset):
    def __init__(self, X, y_G_act, y_G_r, rf_predictions_G_act, rf_predictions_G_r):
        self.X = X
        self.y_G_act = y_G_act
        self.y_G_r = y_G_r
        self.rf_predictions_G_act = rf_predictions_G_act
        self.rf_predictions_G_r = rf_predictions_G_r

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.rf_predictions_G_act[idx], self.rf_predictions_G_r[idx], self.y_G_act[idx], self.y_G_r[idx]

In [5]:
# Read the dataset
df = pd.read_csv('full_dataset.csv') 

# Prepare the feature matrix X and target variables y
X_smiles = df['rxn_smiles']
y_G_act = df['G_act']
y_G_r = df['G_r']

In [6]:
# Define the encoding function
def encode_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
    bitstring = fp.ToBitString()
    features = np.array([int(bit) for bit in bitstring], dtype=np.float32)
    features_tensor = torch.tensor(features)
    return features_tensor

# Encode SMILES strings to fingerprints
X_fingerprints = []

for idx, smiles in enumerate(X_smiles):
    reactants, products = smiles.split('>>')
    reactant_fingerprints = [encode_smiles(reactant) for reactant in reactants.split('.')]
    product_fingerprints = [encode_smiles(product) for product in products.split('.')]
    X_fingerprints.append(reactant_fingerprints + product_fingerprints)

# Convert the list of fingerprints and target values to tensors
X_tensor = torch.stack([torch.cat(fingerprints) for fingerprints in X_fingerprints]).to(device)

# Print the fingerprint tensor
print(X_tensor)
print(X_tensor.shape)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
torch.Size([5269, 3072])


In [7]:
# Convert the data into PyTorch tensors
y_G_act_tensor = torch.tensor(y_G_act.values, dtype=torch.float32).to(device)
y_G_r_tensor = torch.tensor(y_G_r.values, dtype=torch.float32).to(device)

In [8]:
# Split the dataset into training and testing sets
X_train, X_test, y_G_act_train, y_G_act_test, y_G_r_train, y_G_r_test = train_test_split(X_tensor, y_G_act_tensor, y_G_r_tensor, test_size=0.2, random_state=42)


In [9]:
class RandomForestModel(nn.Module):
    def __init__(self, input_size):
        super(RandomForestModel, self).__init__()
        self.random_forest_act = RandomForestRegressor(n_estimators=100)  # Random Forest for G_act
        self.random_forest_r = RandomForestRegressor(n_estimators=100)  # Random Forest for G_r

    def forward(self, x):
        x_act = x  # Input for G_act
        x_r = x  # Input for G_r
        output_act = self.random_forest_act.predict(x_act)  # Output for G_act
        output_r = self.random_forest_r.predict(x_r)  # Output for G_r
        return output_act, output_r

In [10]:

# Instantiate the Random Forest regressor
rf_model_act1 = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model_act2 = RandomForestRegressor(n_estimators=200, random_state=42)
rf_model_act3 = RandomForestRegressor(n_estimators=300, random_state=42)


rf_model_r1 = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model_r2 = RandomForestRegressor(n_estimators=200, random_state=42)
rf_model_r3 = RandomForestRegressor(n_estimators=300, random_state=42)

ensemble_model_G_act = VotingRegressor([('rf_model_act1', rf_model_act1), ('rf_model_act2', rf_model_act2), ('rf_model_act3', rf_model_act3)])
ensemble_model_G_r = VotingRegressor([('rf_model_r1', rf_model_r1), ('rf_model_r2', rf_model_r2), ('rf_model_r3', rf_model_r3)])



In [11]:
# Fit the models
ensemble_model_G_act.fit(X_train, y_G_act_train)
ensemble_model_G_r.fit(X_train, y_G_r_train)

VotingRegressor(estimators=[('rf_model_r1',
                             RandomForestRegressor(random_state=42)),
                            ('rf_model_r2',
                             RandomForestRegressor(n_estimators=200,
                                                   random_state=42)),
                            ('rf_model_r3',
                             RandomForestRegressor(n_estimators=300,
                                                   random_state=42))])

In [12]:
# Get the predictions from the Random Forest model for the training data
rf_predictions_train_G_act = ensemble_model_G_act.predict(X_train)  # G_act
rf_predictions_train_G_r = ensemble_model_G_r.predict(X_train)    # G_r

# Get the predictions from the Random Forest model for the training data
rf_predictions_test_G_act = ensemble_model_G_act.predict(X_test)  # G_act
rf_predictions_test_G_r = ensemble_model_G_r.predict(X_test)    # G_r

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(128, 128)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(128, 64)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.2)
        self.fc4_act = nn.Linear(64, 1)  # Output layer for G_act
        self.fc4_r = nn.Linear(64, 1)  # Output layer for G_r

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.dropout3(x)
        output_act = self.fc4_act(x)  # Output for G_act
        output_r = self.fc4_r(x)  # Output for G_r
        return output_act, output_r

In [15]:
# Instantiate the model
model = NeuralNetwork(X_train.shape[1] + 2)
model.to(device)

NeuralNetwork(
  (fc1): Linear(in_features=3074, out_features=128, bias=True)
  (relu1): ReLU()
  (dropout1): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (relu2): ReLU()
  (dropout2): Dropout(p=0.2, inplace=False)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (relu3): ReLU()
  (dropout3): Dropout(p=0.2, inplace=False)
  (fc4_act): Linear(in_features=64, out_features=1, bias=True)
  (fc4_r): Linear(in_features=64, out_features=1, bias=True)
)

In [16]:
# Define the loss function and optimizer
criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01)

In [17]:
# Define the data loaders
train_dataset = ReactionDataset(X_train, y_G_act_train, y_G_r_train, rf_predictions_train_G_act, rf_predictions_train_G_r)
test_dataset = ReactionDataset(X_test, y_G_act_test, y_G_r_test, rf_predictions_test_G_act, rf_predictions_test_G_r)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16)

In [18]:
# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, rf_predictions_G_act, rf_predictions_G_r, targets_G_act, targets_G_r in train_dataloader:
        targets_G_act = targets_G_act.to(device)
        targets_G_r = targets_G_r.to(device)
        rf_predictions_G_act = rf_predictions_G_act.view(-1, 1).to(device)  # Reshape to match batch size
        rf_predictions_G_r = rf_predictions_G_r.view(-1, 1).to(device)  # Reshape to match batch size
        inputs = inputs.to(device)
        inputs_combined = torch.cat((inputs, rf_predictions_G_act[:, :3072], rf_predictions_G_r[:, :3072]), dim=1)
        inputs_combined = inputs_combined.to(inputs.dtype) 
        optimizer.zero_grad()
        outputs = model(inputs_combined)
        outputs_G_act, outputs_G_r = outputs  # Separate the outputs into two tensors
        loss_G_act = criterion(outputs_G_act, targets_G_act.unsqueeze(1))
        loss_G_r = criterion(outputs_G_r, targets_G_r.unsqueeze(1))
        loss = loss_G_act + loss_G_r
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}")


Epoch 1/500, Loss: 14.616181379014796
Epoch 2/500, Loss: 7.490770972136295
Epoch 3/500, Loss: 6.983305430773533
Epoch 4/500, Loss: 6.695795172091686
Epoch 5/500, Loss: 6.6311501138138045
Epoch 6/500, Loss: 6.442867914835612
Epoch 7/500, Loss: 6.393034766117732
Epoch 8/500, Loss: 6.402597025488362
Epoch 9/500, Loss: 6.266618747602809
Epoch 10/500, Loss: 6.121287361238942
Epoch 11/500, Loss: 6.235463258895007
Epoch 12/500, Loss: 5.99369594093525
Epoch 13/500, Loss: 6.02224322160085
Epoch 14/500, Loss: 6.045827815026948
Epoch 15/500, Loss: 6.066115832690037
Epoch 16/500, Loss: 5.895918401804837
Epoch 17/500, Loss: 6.0128366007949365
Epoch 18/500, Loss: 5.9090795390533675
Epoch 19/500, Loss: 6.055471519629161
Epoch 20/500, Loss: 5.9274172394564655
Epoch 21/500, Loss: 5.983072938341083
Epoch 22/500, Loss: 5.9088052836331455
Epoch 23/500, Loss: 5.884503136078517
Epoch 24/500, Loss: 5.875773319692323
Epoch 25/500, Loss: 5.972414315649957
Epoch 26/500, Loss: 5.824027619578621
Epoch 27/500, Los

In [93]:
# Evaluate the model on the testing data
model.eval()
with torch.no_grad():
    test_loss_G_act = 0.0
    test_loss_G_r = 0.0
    total_mse = 0.0
    total_mae = 0.0
    for inputs, rf_predictions_G_act, rf_predictions_G_r, targets_G_act, targets_G_r in test_dataloader:
        targets_G_act = targets_G_act.to(device)
        targets_G_r = targets_G_r.to(device)
        rf_predictions_G_act = rf_predictions_G_act.view(-1, 1).to(device)  # Reshape to match batch size
        rf_predictions_G_r = rf_predictions_G_r.view(-1, 1).to(device)  # Reshape to match batch size
        inputs = inputs.to(device)
        inputs_combined = torch.cat((inputs, rf_predictions_G_act[:, :3072], rf_predictions_G_r[:, :3072]), dim=1)
        inputs_combined = inputs_combined.to(inputs.dtype) 
        optimizer.zero_grad()
        outputs = model(inputs_combined)
        outputs_G_act, outputs_G_r = outputs  # Separate the outputs into two tensors


        real_G_act = targets_G_act.detach().cpu().numpy()
        real_G_r = targets_G_r.detach().cpu().numpy()

        predictions_G_act = outputs_G_act.detach().cpu().numpy()
        predictions_G_r = outputs_G_r.detach().cpu().numpy()

        print("real_G_act:")
        for real in real_G_act:
            print(real)

        print("real_G_r:")
        for real in real_G_r:
            print(real)

        print("Predictions for G_act:")
        for pred in predictions_G_act:
            print(pred[0])

        print("Predictions for G_r:")
        for pred in predictions_G_r:
            print(pred[0])

        loss_G_act = criterion(outputs_G_act, targets_G_act.unsqueeze(1))
        loss_G_r = criterion(outputs_G_r, targets_G_r.unsqueeze(1))

        test_loss_G_act += loss_G_act.item()
        test_loss_G_r += loss_G_r.item()

        mse_G_act = mean_squared_error(targets_G_act.cpu(), outputs_G_act.cpu().detach())
        mae_G_act = mean_absolute_error(targets_G_act.cpu(), outputs_G_act.cpu().detach())
        mse_G_r = mean_squared_error(targets_G_r.cpu(), outputs_G_r.cpu().detach())
        mae_G_r = mean_absolute_error(targets_G_r.cpu(), outputs_G_r.cpu().detach())

        total_mse += mse_G_act.item() + mse_G_r.item()
        total_mae += mae_G_act.item() + mae_G_r.item()

    average_test_loss_G_act = test_loss_G_act / len(test_dataloader)
    average_test_loss_G_r = test_loss_G_r / len(test_dataloader)

    avg_mse = total_mse / len(test_dataloader)
    avg_mae = total_mae / len(test_dataloader)

    print(f"Average Test Loss G_act: {average_test_loss_G_act}")
    print(f"Average Test Loss G_r: {average_test_loss_G_r}")

    print(f"Mean Squared Error (MSE): {avg_mse:.4f}")
    print(f"Mean Absolute Error (MAE): {avg_mae:.4f}")

real_G_act:
12.93609
6.289673
5.228843
22.114607
27.074554
27.584665
15.497421
21.034805
30.726696
28.207724
15.235269
0.7269611
19.675278
21.075865
20.801397
38.59311
real_G_r:
0.8511472
-86.79967
-46.124702
-31.739902
-21.573524
-12.828893
-41.3358
-5.8700166
6.276678
-1.8251356
-44.857964
-98.36848
-20.077269
-10.068436
-35.77911
3.9598606
Predictions for G_act:
16.446415
7.28142
10.378969
22.870718
22.63221
21.171917
11.175917
20.488361
26.645994
27.583822
14.813046
10.65305
19.592985
17.865582
17.123451
31.101297
Predictions for G_r:
-0.29964638
-77.03292
-32.222992
-29.695507
-24.925655
-22.313309
-43.973995
-15.820522
-5.6808825
-0.54877377
-40.995705
-76.53854
-20.352243
-17.031477
-40.26204
-3.9027035
real_G_act:
25.477125
21.254118
28.825546
30.281055
39.965725
13.976855
17.904808
32.79103
24.780472
22.212963
38.234936
24.284002
23.767738
19.195234
28.22873
19.39669
real_G_r:
-15.1374
-3.9992428
-1.3276277
-11.702599
12.43444
-63.624374
-36.99125
0.16762045
-19.210445
-27.972

# For showing how it works, we will use the following example:

In [85]:

# Read the dataset
df = pd.read_csv('debuging_dataset.csv')

# Extract the SMILES strings
smiles = df['rxn_smiles']

print(f"G_act {df['G_act']} G_r {df['G_r']}  ")

# Encode SMILES strings to fingerprints
X_fingerprints = []

for idx, smiles in enumerate(smiles):
    reactants, products = smiles.split('>>')
    reactant_fingerprints = [encode_smiles(reactant) for reactant in reactants.split('.')]
    product_fingerprints = [encode_smiles(product) for product in products.split('.')]
    X_fingerprints.append(reactant_fingerprints + product_fingerprints)

# Convert the list of fingerprints and target values to tensors
X_test = torch.stack([torch.cat(fingerprints) for fingerprints in X_fingerprints])
X_test = X_test.to(device)
test1 = X_test[1].unsqueeze(0)
test2 = X_test[2].unsqueeze(0)

# Concatenate the tensors
concatenated = torch.cat((X_test, test1, test2), dim=0)




# Set the model to evaluation mode
model.eval()

# Make predictions on test inputs
with torch.no_grad():
    X_test = X_test.view(X_test.size(0), 3074)
    outputs = model(X_test)  # Separate the predictions into two tensors
    outputs_G_act, outputs_G_r = outputs
    
    

# Convert predictions to numpy arrays
predictions_G_act = predictions_G_act.detach().cpu().numpy()
predictions_G_r = predictions_G_r.detach().cpu().numpy()


print("Predictions for G_act:")
for pred in predictions_G_act:
    print(pred[0])

print("Predictions for G_r:")
for pred in predictions_G_r:
    print(pred[0])

G_act 0     15.875896
1     15.155477
2     18.013824
3     23.687079
4     23.438094
5     20.969801
6     11.722625
7      7.268044
8      6.289673
9     16.044475
10    13.604005
Name: G_act, dtype: float64 G_r 0    -51.881526
1    -51.398681
2    -66.822349
3    -62.481289
4    -84.836459
5    -57.785046
6    -63.721331
7    -86.996102
8    -86.799671
9    -50.409197
10   -51.008126
Name: G_r, dtype: float64  


RuntimeError: ignored